# Data Preparation
## Preliminaries

Importing the packages:

In [1]:
import pandas as pd
import boto3
import io
from io import StringIO
import requests
import json
from time import sleep
import math

Importing the python files:

In [2]:
import s3data
import openfigiapi as openFIGI
import permidapi as permID

## ECB data on CSPP holdings

### Read and combine data from several CSV files

In order to access data, configure your Boto3 credentials (AWS Access Key ID, AWS Secret Access Key, Default region name) via Anaconda promt command `aws configure`.

In [3]:
bucket = 's3groupmorocco'

Loading files on CSPP corporate bonds holdings with filenames starting with "CSPPholdings_":

In [4]:
df1 = s3data.read_multiple_csv(bucket, 'data/CSPPholdings_', 'iso8859-1')

Loading files on CSPP corporate bonds holdings with filenames starting with "CSPP_PEPP_corporate_bond_holdings_":

In [5]:
df2 = s3data.read_multiple_csv(bucket, 'data/CSPP_PEPP_corporate_bond_holdings_', 'iso8859-1')

Combining two dataframes together:

In [6]:
df = pd.concat([df1, df2], ignore_index=True)

Check for the null values:

In [7]:
df.isna().sum()

NCB                2069
ISIN_CODE         21215
ISSUER_NAME_      56389
MATURITY_DATE_    56389
COUPON_RATE_      56389
MONTH                 0
ISSUER_NAME       25409
MATURITY_DATE     25409
COUPON_RATE       25409
Unnamed: 0        59371
ISIN              40226
ISSUER            40226
MATURITY DATE     40226
COUPON RATE       40226
dtype: int64

Column names are a slightly different in reports generated in different months. Filling in empty cells to put all relevant data into one column:

In [8]:
df['NCB'].fillna(df['Unnamed: 0'], inplace=True)
df['ISIN'].fillna(df['ISIN_CODE'], inplace=True)
df['ISSUER'].fillna(df['ISSUER_NAME_'], inplace=True)
df['ISSUER'].fillna(df['ISSUER_NAME'], inplace=True)
df['MATURITY DATE'].fillna(df['MATURITY_DATE_'], inplace=True)
df['MATURITY DATE'].fillna(df['MATURITY_DATE'], inplace=True)
df['COUPON RATE'].fillna(df['COUPON_RATE_'], inplace=True)
df['COUPON RATE'].fillna(df['COUPON_RATE'], inplace=True)

Choosing necessary columns and rows:

In [9]:
df = df[df['ISIN'].notna()][['MONTH', 'NCB', 'ISIN', 'ISSUER', 'MATURITY DATE', 'COUPON RATE']]

Check for the null values:

In [10]:
df.isna().sum()

MONTH            0
NCB              0
ISIN             0
ISSUER           0
MATURITY DATE    0
COUPON RATE      0
dtype: int64

In [11]:
df.head()

,MONTH,NCB,ISIN,ISSUER,MATURITY DATE,COUPON RATE
0,2017/06,IT,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75
1,2017/06,IT,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3
2,2017/06,IT,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125
3,2017/06,IT,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75
4,2017/06,IT,XS0859920406,A2A S.p.A.,28/11/2019,4.5


## Data Extraction via API

Accessing APIs keys and URLs:

In [12]:
keys = s3data.read_csv(bucket,'data/keys.csv')
links = s3data.read_csv(bucket,'data/links.csv')

### OpenFIGI API

We use openFIGI API to map formatted companies' names via ISIN code of the bonds.

Accessing openFIGI API key and URL:

In [13]:
openFIGI_key = keys[keys['API']=='openFIGI']['KEY'].reset_index(drop=True)[0]
openFIGI_url = links[links['API']=='openFIGI']['URL'].reset_index(drop=True)[0]

Creating list of unique ISIN codes:

In [14]:
ISIN_codes = df['ISIN'].drop_duplicates().reset_index(drop=True)

Mapping request:

In [15]:
map_result = openFIGI.map_ISIN(openFIGI_key,openFIGI_url,ISIN_codes)

Merge mapped names to the main dataframe via ISIN:

In [16]:
df = df.merge(pd.concat([ISIN_codes, map_result], axis=1),on='ISIN', how='left')

In [17]:
df.head()

,MONTH,NCB,ISIN,ISSUER,MATURITY DATE,COUPON RATE,name
0,2017/06,IT,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75,2I RETE GAS SPA
1,2017/06,IT,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3,2I RETE GAS SPA
2,2017/06,IT,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125,2I RETE GAS SPA
3,2017/06,IT,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75,2I RETE GAS SPA
4,2017/06,IT,XS0859920406,A2A S.p.A.,28/11/2019,4.5,A2A SPA


### PermID Record Matching API

We use different PermID APIs to assign Industrial groups and Business sectors to companies. Accessing token for API requests:

In [18]:
permID_token = keys[keys['API']=='permID']['KEY'].reset_index(drop=True)[0]

Accessing PermID Record Matching API URL:

In [19]:
permIDmatch_url = links[links['API']=='permID Record Matching']['URL'].reset_index(drop=True)[0]

Creating a csv template for Record Matching API:

Instructions on the format of the template can be found here: https://permid.org/match.

In [20]:
df['name'] = df['name'].str.split().str[:1].str.join(' ') + '|' + \
             df['name'].str.split().str[:2].str.join(' ') + '|' + df['name']

In [21]:
template = df['name'].drop_duplicates().to_csv(index=False)

Find companies' PermIDs by their names using Record Matching API:

In [22]:
match_results = permID.record_matching(permID_token,permIDmatch_url,template)

Processed: 393
Matched: 
  Total 351
  Excellent 296
  Good 22
  Possible 33
Unmatched: 42


<b>Note:</b> Sometimes the request is processed successfully, but does not return any matches due to unexpected server errors. Then the message appears and the request is sent again.

Around 90% of the companies from the list were matched. Merge PermIDs to the main dataframe via template name:

In [23]:
df = df.merge(match_results, left_on = 'name', right_on = 'Input_Name', how = 'left').drop(columns='Input_Name')
df.head()

,MONTH,NCB,ISIN,ISSUER,MATURITY DATE,COUPON RATE,name,Match OpenPermID
0,2017/06,IT,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75,2I|2I RETE|2I RETE GAS SPA,https://permid.org/1-5000936840
1,2017/06,IT,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3,2I|2I RETE|2I RETE GAS SPA,https://permid.org/1-5000936840
2,2017/06,IT,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125,2I|2I RETE|2I RETE GAS SPA,https://permid.org/1-5000936840
3,2017/06,IT,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75,2I|2I RETE|2I RETE GAS SPA,https://permid.org/1-5000936840
4,2017/06,IT,XS0859920406,A2A S.p.A.,28/11/2019,4.5,A2A|A2A SPA|A2A SPA,https://permid.org/1-5000005309


Around 10% of the data is not matched via names mapped from openFIGI. We try another approach.   
Creating a template from raw companies' names:

We decode special letters to their English equivalent, remove special characters, additional spaces, etc. Then we create a new csv template only for companies that do not have PermID.

In [24]:
df['NAME'] = df['ISSUER'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').\
                                    str.replace('.',' ').str.replace('/',' ').str.replace('-',' ').str.replace('  ',' ').\
                                    str.replace('(','').str.replace(')','').str.split(',',1).str[0].str.strip().str.lower()     
df['NAME'] =  df['NAME'].str.split().str[:1].str.join(' ') + '|' + \
              df['NAME'].str.split().str[:2].str.join(' ') + '|' + df['NAME']

In [25]:
template = df[df['Match OpenPermID'].isna()]['NAME'].drop_duplicates().to_csv(index=False)

Find companies' PermIDs using Record Matching API:

In [26]:
match_results = permID.record_matching(permID_token,permIDmatch_url,template)

Processed: 50
Matched: 
  Total 35
  Excellent 28
  Good 5
  Possible 2
Unmatched: 15


Only several companies are not matched. Updating the main datafraime with additionally matched PermIDs:

In [27]:
df.update(df.drop(columns=['Match OpenPermID']).merge(match_results[match_results['Match OpenPermID'].notna()],\
                                                      left_on = 'NAME', right_on = 'Input_Name', how = 'left'))

### PermID Entity Search API

To find PermID for the rest of the companies, we use PermID Entity Search API. Accessing API URL:

In [28]:
permIDsearch_url = links[links['API']=='permID Entity Search']['URL'].reset_index(drop=True)[0]

List of possible queries for each company using columns generated for Record Matching API:

In [29]:
df['SEARCH'] = df['name'] + '|' + df['NAME'] + '|' + df['ISSUER']

 - We make reqests with several search queries (first word, first two words, whole name) via Entity Search API for each company that does not have PermID;  
 - Then we append results of each request to one dataframe;  
 - From the obtained list we choose the best search result for each company and get corresponding permID.

In [30]:
res = []
for name in df[df['Match OpenPermID'].isna()]['SEARCH'].drop_duplicates():
    name_results = []
    for query in name.split('|'):
        search_company = permID.entity_search(permID_token,permIDsearch_url,query)
        if search_company['total'] != 0:
            name_results = name_results + search_company['entities']
    res.append(pd.DataFrame(name_results).assign(ISSUER = name.split('|')[6]))

<b>Note:</b> In case of an exception the message appears, the request is sent again after a pause.

In [31]:
res = pd.concat(res,ignore_index=True).drop_duplicates()

Display results:

In [32]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(res[['ISSUER','organizationName']].drop_duplicates())

,ISSUER,organizationName
0,Autostr. Bres.Ver.Vic.Pad. SpA,ASTM SpA
1,Autostr. Bres.Ver.Vic.Pad. SpA,Societa di Progetto Autostrada Diretta Brescia...
2,Autostr. Bres.Ver.Vic.Pad. SpA,A4 Holding SpA
3,Autostr. Bres.Ver.Vic.Pad. SpA,ATIVA Autostrada Torino Ivrea Valle d'Aosta SpA
4,Autostr. Bres.Ver.Vic.Pad. SpA,Autostrada Albenga Garessio Ceva SpA
7,Autostr. Bres.Ver.Vic.Pad. SpA,Autostrada Brescia Verona Vicenza Padova SpA
8,Autostr. Bres.Ver.Vic.Pad. SpA,Autostrada Campogalliano Sassuolo SpA
9,Autostr. Bres.Ver.Vic.Pad. SpA,Autostrada del Molise SpA
10,Autostr. Bres.Ver.Vic.Pad. SpA,Mattioda Autostrade SpA
11,Ciba Spc. Chem. Fin. Lxbg S.A.,CIBA SPECIAL-S


Next results are chosen:

In [33]:
org = ['Autostrada Brescia Verona Vicenza Padova SpA','CIBA Corp','Distribuidora Internacional de Alimentacion SA', \
       'Elia Group SA','O2 Telefonica Deutschland Finanzierungs GmbH','Red Electrica Financiaciones SAU',\
       'Resa SA','Societe Nationale SNCF SA','Unilever PLC','Vivendi SE','Covivio Hotels SCA',\
       'Mondelez International Holdings Netherlands BV','Elo SA']
res = res[res.organizationName.isin(org)][['@id','ISSUER']].rename(columns={'@id':'Match OpenPermID'})

Updating the main datafraime with additionally matched PermIDs:

In [34]:
df.update(df.drop(columns=['Match OpenPermID']).merge(res, on='ISSUER', how = 'left'))

### PermID Entity Lookup API

Next we have to get the data on industry for each company and join it to the main dataframe. A list of PermIDs for all companies:

In [35]:
IDs = df[df['Match OpenPermID'].notna()]['Match OpenPermID'].drop_duplicates()

 - We request information about the company by its PermID using Entity Lookup API;  
 - If Industry Group key is available in the response, we request information about Industry Group and Business Sector using their PermIDs;  
 - We append data into one dataframe.

<b>Note</b>: for one Industry Group the label value was returned as a list (['Freight&Logistics Services', 'Freight & Logistics Services']), so additional check for the type is added.

In [36]:
sectors = []
for ID in IDs:
    lookup_company = permID.entity_lookup(permID_token, ID)
    if 'hasPrimaryIndustryGroup' in lookup_company:
        industryID = lookup_company['hasPrimaryIndustryGroup']
        sectorID = lookup_company['hasPrimaryBusinessSector']
        lookup_industry = permID.entity_lookup(permID_token, industryID)
        lookup_sector = permID.entity_lookup(permID_token, sectorID)
        industry = lookup_industry['prefLabel'][0] if type(lookup_industry['prefLabel']) == list \
                                                    else lookup_industry['prefLabel']
        sectors.append(pd.DataFrame({'Match OpenPermID' : [ID],
                                     'Industry group ID' : [industryID],
                                     'Industry Group' : [industry],
                                     'Business Sector' : [lookup_sector['prefLabel']]}))

In [37]:
sectors = pd.concat(sectors,ignore_index=True).drop_duplicates()
sectors.head()

,Match OpenPermID,Industry group ID,Industry Group,Business Sector
0,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
1,https://permid.org/1-5000005309,https://permid.org/1-4294952819,Electric Utilities & IPPs,Utilities
2,https://permid.org/1-5000066931,https://permid.org/1-4294952765,"Machinery, Tools, Heavy Vehicles, Trains & Ships",Industrial Goods
3,https://permid.org/1-4295889666,https://permid.org/1-4294952750,Transport Infrastructure,Transportation
4,https://permid.org/1-4295875677,https://permid.org/1-4294952813,Multiline Utilities,Utilities


Add data about companies' Industry Groups and Business Sectors to the main dataframe by Match OpenPermID:

In [38]:
df = df.merge(sectors, how='left').drop(columns=['name','NAME','SEARCH'])
df.head()

,MONTH,NCB,ISIN,ISSUER,MATURITY DATE,COUPON RATE,Match OpenPermID,Industry group ID,Industry Group,Business Sector
0,2017/06,IT,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
1,2017/06,IT,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
2,2017/06,IT,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
3,2017/06,IT,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75,https://permid.org/1-5000936840,https://permid.org/1-4294952817,Natural Gas Utilities,Utilities
4,2017/06,IT,XS0859920406,A2A S.p.A.,28/11/2019,4.5,https://permid.org/1-5000005309,https://permid.org/1-4294952819,Electric Utilities & IPPs,Utilities


In [39]:
df.isna().sum()

MONTH                   0
NCB                     0
ISIN                    0
ISSUER                  0
MATURITY DATE           0
COUPON RATE             0
Match OpenPermID      196
Industry group ID    5678
Industry Group       5678
Business Sector      5678
dtype: int64

Put a resulting dataframe into "CSPP_bonds_with_sectors.csv" file on S3 bucket to use it in [further analysis](ECB%20bond%20purchases.ipynb):

In [40]:
s3data.df_to_csv(df, bucket, 'data/CSPP_bonds_with_sectors.csv')